In [1]:
#Full Code

from pdf2image import convert_from_path
import os
import sys

def convert_pdf_to_jpeg_with_input():
    """
    사용자 입력을 받아 PDF 파일을 JPEG 이미지로 변환하는 함수.
    """
    print("-" * 50)
    print("PDF를 JPEG 이미지로 변환합니다.")
    print("-" * 50)

    # 1. PDF 파일 경로 입력 받기
    # Raw String으로 처리하여 백슬래시(\) 경로 오류 방지
    pdf_path = input("✅ 변환할 PDF 파일의 전체 경로(파일명 포함)를 입력하세요: ")
    pdf_path = pdf_path.strip().strip('"').strip("'") # 앞뒤 공백과 따옴표 제거

    # 2. 이미지 저장 폴더 경로 입력 받기
    output_dir = input("✅ 이미지를 저장할 폴더의 전체 경로를 입력하세요: ")
    output_dir = output_dir.strip().strip('"').strip("'") # 앞뒤 공백과 따옴표 제거

    # 3. Poppler bin 폴더 경로 입력 받기
    # 이 경로는 Poppler 설치 위치에 따라 다릅니다.
    #poppler_path_input = input("✅ Poppler 설치 폴더의 bin 경로를 입력하세요 (예: C:\\...\\poppler-xxx\\bin): ")
    #poppler_path = poppler_path_input.strip().strip('"').strip("'")
    poppler_path = r"C:\Users\DJ-OIG-KWONSH\AppData\Local\miniconda3\Lib\site-packages\poppler-25.12.0\Library\bin"


    # --- 입력된 경로 유효성 검사 ---
    if not os.path.exists(pdf_path):
        print(f"\n❌ 오류: PDF 파일을 찾을 수 없습니다: {pdf_path}")
        return
    
    if not os.path.exists(output_dir):
        print(f"\n❌ 오류: 저장 폴더를 찾을 수 없습니다. 폴더를 생성하거나 경로를 확인하세요: {output_dir}")
        return

    # PDF 파일명에서 확장자를 제외한 기본 이름을 추출
    base_name = os.path.splitext(os.path.basename(pdf_path))[0]
    
    print("-" * 50)
    print(f"PDF 파일: {pdf_path}")
    print(f"저장 폴더: {output_dir}")
    print(f"Poppler 경로: {poppler_path}")
    print("-" * 50)

    try:
        print("💡 변환 시작... (대용량 파일은 시간이 걸릴 수 있습니다.)")
        # convert_from_path를 이용해 pdf파일를 이미지형태로 불러옵니다.
        images = convert_from_path(
            pdf_path, 
            poppler_path=poppler_path,
            # 이미지 품질 설정을 위한 dpi (선택 사항, 기본 100)
            dpi=200 
        )
        
        # 각각의 페이지(이미지)를 jpg로 저장
        for i, image in enumerate(images):
            # 파일명을 [PDF_기본이름]_[페이지번호].jpg 형식으로 저장
            filename = f"{base_name}_{i+1}.jpg"
            output_file_path = os.path.join(output_dir, filename)
            
            # JPEG 형식으로 저장
            image.save(output_file_path, "JPEG")
            print(f"✅ 페이지 {i+1} 저장 완료: {output_file_path}")
            
        print('\n🌟 Conversion Complete: PDF 파일이 성공적으로 JPEG 이미지로 변환되었습니다.')
        
    except Exception as e:
        print(f"\n❌ 치명적인 오류 발생: {e}")
        if 'No such file or directory' in str(e) or 'is not recognized' in str(e):
             print("\n[오류 해결 팁]")
             print("1. Poppler의 'bin' 폴더 경로를 정확하게 입력했는지 확인하세요.")
             print("2. 해당 'bin' 폴더 안에 'pdftoppm.exe' 파일이 있는지 확인하세요.")
        elif 'PDF info is invalid' in str(e):
             print("3. 입력한 PDF 파일이 손상되지 않았는지 확인하세요.")

if __name__ == "__main__":
    convert_pdf_to_jpeg_with_input()

# 이미지 파일에서 OCR
import pytesseract
from PIL import Image
import os
from glob import glob
import re # 정규표현식 모듈 추가
# ----------------------------------------------------
# ⚠️ Tesseract 경로 설정 (필요한 경우 주석 해제 후 수정)
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
# ----------------------------------------------------
pytesseract.pytesseract.tesseract_cmd = r"C:\Users\DJ-OIG-KWONSH\AppData\Local\Programs\Tesseract-OCR\tesseract.exe"

def ocr_from_image(image_path, lang='kor+eng'):
    """
    이미지 파일에서 OCR을 수행하여 텍스트를 추출합니다.
    
    Args:
        image_path (str): OCR을 수행할 이미지 파일의 경로.
        lang (str): 사용할 언어 코드 (예: 'eng', 'kor', 'kor+eng'). 
                    언어 팩은 Tesseract 설치 시 함께 설치되어 있어야 합니다.
    """
    if not os.path.exists(image_path):
        return f"오류: 이미지 파일을 찾을 수 없습니다. 경로를 확인하세요: {image_path}", None
        
    try:
        img = Image.open(image_path)
        # OCR 수행 및 텍스트 추출
        text = pytesseract.image_to_string(img, lang=lang)
        
        # 성공 메시지와 추출된 텍스트 반환
        return "SUCCESS", text.strip()

    except pytesseract.TesseractNotFoundError:
        return "오류: Tesseract OCR 엔진이 설치되지 않았거나 경로를 찾을 수 없습니다. Tesseract 설치를 확인하세요.", None
    except Exception as e:
        return f"OCR 처리 중 오류 발생: {e}", None


def process_multiple_images(directory_path, lang='kor+eng'):
    """
    지정된 폴더 내의 모든 jpg 파일에 대해 OCR을 수행하고 결과를 개별 파일로 저장합니다.
    """
    if not os.path.isdir(directory_path):
        print(f"\n❌ 오류: 유효하지 않은 폴더 경로입니다: {directory_path}")
        return

    print(f"\n--- 폴더 [{directory_path}] 내 모든 jpg 파일 처리 시작 ---")
    
    # 1. 디렉토리 내에서 확장자가 '.jpg'인 파일 목록을 가져옵니다.
    jpg_files = [f for f in os.listdir(directory_path) if f.lower().endswith('.jpg')]
    
    if not jpg_files:
        print("💡 해당 폴더에서 jpg 파일을 찾을 수 없습니다.")
        return

    total_files = len(jpg_files)
    print(f"총 {total_files} 개의 jpg 파일이 발견되었습니다.")
    
    # 텍스트 파일 저장 성공 여부 플래그
    extraction_successful = False 

    # 2. 각 파일을 반복 처리
    for index, filename in enumerate(jpg_files):
        image_path = os.path.join(directory_path, filename)
        print(f"\n[{index + 1}/{total_files}] 파일 처리 중: {filename}")
        
        status, extracted_content = ocr_from_image(image_path, lang=lang)
        
        if status == "SUCCESS":
            if extracted_content:
                print("✅ 텍스트 추출 성공. 저장 중...")
                
                # 텍스트 파일 저장
                try:
                    base_name = os.path.splitext(filename)[0]
                    output_filename = base_name + "_extracted.txt"
                    output_path = os.path.join(directory_path, output_filename)
                    
                    with open(output_path, 'w', encoding='utf-8') as f:
                        f.write(extracted_content)
                    
                    print(f"   -> 저장 완료: {output_filename}")
                    extraction_successful = True # 저장 성공 플래그 설정
                    
                except Exception as e:
                    print(f"❌ 파일 저장 중 오류 발생: {e}")
            else:
                print("⚠️ 텍스트는 추출되었으나 내용이 비어 있습니다.")
        else:
            print(f"❌ OCR 실패: {status}")

    print("\n--- 개별 파일 처리 완료 ---")
    
    # 개별 파일 추출이 하나라도 성공했을 경우에만 통합 함수 호출
    if extraction_successful:
        combine_text_files(directory_path)


def combine_text_files(directory_path):
    """
    지정된 폴더 내의 모든 텍스트 파일(*_extracted.txt)을 파일명에 포함된 숫자를 기준으로
    (자연 정렬) 하나의 파일로 통합합니다.
    """
    
    # glob을 사용하여 *_extracted.txt 파일 목록을 가져옵니다.
    # 해당 패턴과 정확히 일치하는 파일만 선택합니다.
    txt_files = glob(os.path.join(directory_path, "*_extracted.txt"))
    
    if not txt_files:
        print("💡 통합할 텍스트 파일(*_extracted.txt)을 찾을 수 없습니다.")
        return
        
    # 통합 파일 이름 지정
    combined_file_path = os.path.join(directory_path, "combined_ocr_output.txt")
    
    print("\n--- 텍스트 파일 통합 작업 시작 (자연 정렬 적용) ---")
    
    # -----------------------------------------------------------------------
    # 1. 파일명에서 숫자 부분을 추출하는 정규표현식
    # 패턴: 마지막 밑줄('_') 뒤에 오는 숫자 (예: ..._1_extracted.txt 에서 '1')
    def natural_sort_key(s):
        # s는 파일의 전체 경로입니다. os.path.basename으로 파일명만 추출
        filename = os.path.basename(s)
        # 밑줄('_')로 분리된 문자열에서 숫자 부분만 추출
        parts = re.split(r'(\d+)', filename)
        
        # 숫자 부분은 int로, 문자열 부분은 그대로 반환하여 정렬 키로 사용
        return [int(text) if text.isdigit() else text for text in parts]

    # 2. natural_sort_key 함수를 기준으로 정렬
    sorted_txt_files = sorted(txt_files, key=natural_sort_key)
    # -----------------------------------------------------------------------
    
    try:
        with open(combined_file_path, 'w', encoding='utf-8') as outfile:
            # 정렬된 목록을 순회
            for filepath in sorted_txt_files: 
                filename = os.path.basename(filepath)
                
                # 파일 시작 구분자 추가
                #outfile.write(f"\n\n==================== [START:] ====================\n\n")
                outfile.write(f"\n\n====================n\n")
                
                # 개별 파일 내용 읽기 및 쓰기
                with open(filepath, 'r', encoding='utf-8') as infile:
                    outfile.write(infile.read())
                    
                # 파일 끝 구분자 추가
                #outfile.write(f"\n\n===================== [END:] =====================\n\n")
                outfile.write(f"\n\n====================n\n")
                print(f"   -> 통합 완료: {filename}")
                
        print(f"\n🎉 모든 텍스트 파일이 성공적으로 통합되었습니다!")
        print(f"   -> 통합 파일 경로: {combined_file_path}")
        
    except Exception as e:
        print(f"❌ 텍스트 파일 통합 중 오류 발생: {e}")


# --- 메인 실행 부분 ---
if __name__ == "__main__":
    
    # ⚠️ 1. # OCR을 수행할 jpg 파일들이 있는 폴더의 절대 경로를 입력받습니다.
    # 예: r"C:\Users\User\Documents\ScanImages"        
    image_directory = input("✅ OCR을 수행할 jpg 파일들이 있는 폴더의 경로를 입력하세요: ")
    
    # 추출 언어 지정
    language = 'kor+eng'

    # 모든 이미지 처리 및 통합 함수 실행
    process_multiple_images(image_directory, language)

--------------------------------------------------
PDF를 JPEG 이미지로 변환합니다.
--------------------------------------------------


✅ 변환할 PDF 파일의 전체 경로(파일명 포함)를 입력하세요:  C:\KBK_DownloadFile\IP-2025-0123-SE-000\IP-2025-0123-SE-000.pdf
✅ 이미지를 저장할 폴더의 전체 경로를 입력하세요:  C:\KBK_DownloadFile\IP-2025-0123-SE-000


--------------------------------------------------
PDF 파일: C:\KBK_DownloadFile\IP-2025-0123-SE-000\IP-2025-0123-SE-000.pdf
저장 폴더: C:\KBK_DownloadFile\IP-2025-0123-SE-000
Poppler 경로: C:\Users\DJ-OIG-KWONSH\AppData\Local\miniconda3\Lib\site-packages\poppler-25.12.0\Library\bin
--------------------------------------------------
💡 변환 시작... (대용량 파일은 시간이 걸릴 수 있습니다.)
✅ 페이지 1 저장 완료: C:\KBK_DownloadFile\IP-2025-0123-SE-000\IP-2025-0123-SE-000_1.jpg
✅ 페이지 2 저장 완료: C:\KBK_DownloadFile\IP-2025-0123-SE-000\IP-2025-0123-SE-000_2.jpg
✅ 페이지 3 저장 완료: C:\KBK_DownloadFile\IP-2025-0123-SE-000\IP-2025-0123-SE-000_3.jpg
✅ 페이지 4 저장 완료: C:\KBK_DownloadFile\IP-2025-0123-SE-000\IP-2025-0123-SE-000_4.jpg
✅ 페이지 5 저장 완료: C:\KBK_DownloadFile\IP-2025-0123-SE-000\IP-2025-0123-SE-000_5.jpg
✅ 페이지 6 저장 완료: C:\KBK_DownloadFile\IP-2025-0123-SE-000\IP-2025-0123-SE-000_6.jpg
✅ 페이지 7 저장 완료: C:\KBK_DownloadFile\IP-2025-0123-SE-000\IP-2025-0123-SE-000_7.jpg
✅ 페이지 8 저장 완료: C:\KBK_DownloadFile\IP-2025-0123-SE-000\IP-2025-0123-SE

✅ OCR을 수행할 jpg 파일들이 있는 폴더의 경로를 입력하세요:  C:\KBK_DownloadFile\IP-2025-0123-SE-000



--- 폴더 [C:\KBK_DownloadFile\IP-2025-0123-SE-000] 내 모든 jpg 파일 처리 시작 ---
총 32 개의 jpg 파일이 발견되었습니다.

[1/32] 파일 처리 중: IP-2025-0123-SE-000_1.jpg
✅ 텍스트 추출 성공. 저장 중...
   -> 저장 완료: IP-2025-0123-SE-000_1_extracted.txt

[2/32] 파일 처리 중: IP-2025-0123-SE-000_10.jpg
✅ 텍스트 추출 성공. 저장 중...
   -> 저장 완료: IP-2025-0123-SE-000_10_extracted.txt

[3/32] 파일 처리 중: IP-2025-0123-SE-000_11.jpg
✅ 텍스트 추출 성공. 저장 중...
   -> 저장 완료: IP-2025-0123-SE-000_11_extracted.txt

[4/32] 파일 처리 중: IP-2025-0123-SE-000_12.jpg
✅ 텍스트 추출 성공. 저장 중...
   -> 저장 완료: IP-2025-0123-SE-000_12_extracted.txt

[5/32] 파일 처리 중: IP-2025-0123-SE-000_13.jpg
✅ 텍스트 추출 성공. 저장 중...
   -> 저장 완료: IP-2025-0123-SE-000_13_extracted.txt

[6/32] 파일 처리 중: IP-2025-0123-SE-000_14.jpg
✅ 텍스트 추출 성공. 저장 중...
   -> 저장 완료: IP-2025-0123-SE-000_14_extracted.txt

[7/32] 파일 처리 중: IP-2025-0123-SE-000_15.jpg
✅ 텍스트 추출 성공. 저장 중...
   -> 저장 완료: IP-2025-0123-SE-000_15_extracted.txt

[8/32] 파일 처리 중: IP-2025-0123-SE-000_16.jpg
✅ 텍스트 추출 성공. 저장 중...
   -> 저장 완료: IP-2025-0123-SE-000_16